In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['PATH_TO_ENV'] = "~/projects/chatsky-llm-autoconfig/.env"
os.getenv('EMBEDDER_MODEL')

'BAAI/bge-m3'

In [3]:
from dialogue2graph.pipelines.core.dialogue import DialogueMessage, Dialogue
import pickle
import numpy as np

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


# Naive Augmentation

In [90]:
with open("../data/gen_dataset_augment_naive", "rb") as fp:
    data = pickle.load(fp)

In [44]:
len(data)

160

In [ ]:
from tqdm import tqdm
from dialogue2graph.metrics.no_llm_metrics.metrics import match_roles, is_correct_length
from augmentation_utils import (
    check_no_duplicates_all_dialogues,
    check_no_duplicates_one_dialogue
)

### Checking duplicates within dialogues per graph

In [64]:
new_data = []
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    print(f'example {i}')
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        if not check_no_duplicates_all_dialogues(augmented_dialogues):
            failure_examples.append(i)
        else:
            new_data.append(example)
    except Exception as e:
        errors.append((i, e))

100%|██████████| 160/160 [00:00<00:00, 14610.16it/s]

example 0
common_elements: {"Alright, don't hesitate to reach out if you need anything else. Have a wonderful day!", 'Sure, it’s 123 Main St.', 'Would you prefer that in red or blue?', 'Hello! How can I help you today?', "I'm looking to place an order."}
example 1
common_elements: {'Have you attempted to reset your password yet?', 'Thanks for your help.', "I'm unable to get into my profile.", 'Thank you for contacting us. Wishing you a wonderful day ahead!', "I can't access my account.", 'Yes, please go ahead and escalate it.', "Hi there! I'm really sorry to hear you're having issues. How can I help you today?"}
example 2
common_elements: {'Hi there! How can I help you today?', 'Got it, thank you for the advice.'}
example 3
example 4
common_elements: {'Hi there! How can I help you today?'}
example 5
common_elements: {'Hi there! Thank you for joining our recycling initiative. How can I help you today?', 'Hi, could you explain how the recycling program functions?'}
example 6
example 7
co

In [47]:
# duplicates
print(failure_examples)

[0, 1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159]


In [48]:
# generation errors
errors

[(6, TypeError("'NoneType' object is not iterable")),
 (30, TypeError("'NoneType' object is not iterable")),
 (66, TypeError("'NoneType' object is not iterable")),
 (74, TypeError("'NoneType' object is not iterable"))]

In [63]:
for example in [data[6], data[30], data[66], data[74]]:
    dialogues = [dial['messages'] for dial in example['dialogues']]
    print(len(dialogues))
    lens = [len(dia) for dia in dialogues]
    print(lens)

5
[15, 15, 15, 15, 15]
7
[11, 11, 11, 11, 9, 7, 7]
4
[19, 19, 13, 11]
10
[13, 13, 13, 13, 13, 13, 11, 9, 7, 7]


In [49]:
len(new_data)

7

### Checking duplicates within one dialogue

In [66]:
errors = []
failure_examples = []
new_data_2 = []

for i, example in enumerate(tqdm(new_data)):
    print(f'example {i}')
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    for j, aug_dia in enumerate(augmented_dialogues):
        try:
            if not check_no_duplicates_one_dialogue(aug_dia):
                failure_examples.append((i, j))
            else:
                true_count += 1
        except Exception as e:
            errors.append((i, j, e))
    if true_count == len(augmented_dialogues):
        new_data_2.append(example)

100%|██████████| 7/7 [00:00<00:00, 10500.76it/s]

example 0
example 1
example 2
example 3
common_elements: ['Have you verified that your internet connection is stable?', 'Yes, my internet connection is stable.']
common_elements: ['Have you verified that your internet connection is stable?', 'Yes, my internet connection seems fine.', "That's good to know. Let's try shutting down any applications that might be consuming bandwidth. Please do that now."]
example 4
example 5
example 6


In [67]:
print(len(failure_examples))
print(failure_examples)

2
[(3, 2), (3, 3)]


In [68]:
errors

[]

In [69]:
len(new_data_2)

6

### Checking matching roles and length correctness

In [70]:
errors = []
failure_examples = []
new_data_3 = []

for i, example in enumerate(tqdm(new_data_2)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if not match_roles(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                failure_examples.append((i, j))
            else:
                true_count += 1
    except Exception as e:
        errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data_3.append(example)

100%|██████████| 6/6 [00:00<00:00, 2854.24it/s]


In [71]:
errors = []
failure_examples = []
new_data_4 = []

for i, example in enumerate(tqdm(new_data_3)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if not is_correct_length(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                failure_examples.append((i, j))
            else:
                true_count += 1
    except Exception as e:
        errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data_4.append(example)

100%|██████████| 6/6 [00:00<00:00, 3359.03it/s]


In [73]:
len(new_data_4)

6

In [ ]:
with open("../data/gen_dataset_augment_naive_filtered", "wb") as fp:
    pickle.dump(new_data_4, fp)

# One-shot Augmentation

In [ ]:
with open("../data/gen_dataset_augment_one-shot", "rb") as fp:
    data_1 = pickle.load(fp)
with open("../data/gen_dataset_augment_one-shot_2", "rb") as fp:
    data_2 = pickle.load(fp)
data = data_1 + data_2
len(data)

4

In [80]:
from tqdm import tqdm
from dialogue2graph.metrics.no_llm_metrics.metrics import match_roles, is_correct_length
from augmentation_utils import (
    check_no_duplicates_all_dialogues,
    check_no_duplicates_one_dialogue
)

### Checking duplicates within dialogues per graph

In [81]:
new_data = []
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    print(f'example {i}')
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        if not check_no_duplicates_all_dialogues(augmented_dialogues):
            failure_examples.append(i)
        else:
            new_data.append(example)
    except Exception as e:
        errors.append((i, e))

100%|██████████| 4/4 [00:00<00:00, 8706.39it/s]

example 0
common_elements: {'What size do you need?', 'Hey there! How can I help you today?', 'Sure, it’s 123 Main St.', 'Medium, please.', 'Red, please.'}
example 1
common_elements: {"I'm unable to access my account.", 'Have you attempted to reset your password?', 'Thanks for reaching out! Have a wonderful day!', 'Thanks for your help.'}
example 2
example 3


In [82]:
# duplicates
print(failure_examples)
# generation errors
errors

[0, 1]


[(2, TypeError("'OutputParserException' object is not iterable"))]

### Checking duplicates within one dialogue

In [84]:
errors = []
failure_examples = []
new_data_2 = []

for i, example in enumerate(tqdm(new_data)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    for j, aug_dia in enumerate(augmented_dialogues):
        try:
            if not check_no_duplicates_one_dialogue(aug_dia):
                failure_examples.append((i, j))
            else:
                true_count += 1
        except Exception as e:
            errors.append((i, j, e))
    if true_count == len(augmented_dialogues):
        new_data_2.append(example)

100%|██████████| 1/1 [00:00<00:00, 7667.83it/s]


### Checking matching roles and length correctness

In [86]:
errors = []
failure_examples = []
new_data_3 = []

for i, example in enumerate(tqdm(new_data_2)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if not match_roles(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                failure_examples.append((i, j))
            else:
                true_count += 1
    except Exception as e:
        errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data_3.append(example)

100%|██████████| 1/1 [00:00<00:00, 2169.84it/s]


In [87]:
errors = []
failure_examples = []
new_data_4 = []

for i, example in enumerate(tqdm(new_data_3)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if not is_correct_length(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                failure_examples.append((i, j))
            else:
                true_count += 1
    except Exception as e:
        errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data_4.append(example)

100%|██████████| 1/1 [00:00<00:00, 2391.28it/s]
